In [1]:
import choclo
import harmonica as hm
import numba
import numpy as np
import verde as vd

from magali._inversion import MagneticMomentBz
from magali._synthetic import dipole_bz
from magali._units import (
    coordinates_micrometer_to_meter,
)

from magali._validation import check_fit_input

In [2]:
dipole_coordinates = (500, 500, -15)
true_inclination = 30
true_declination = 40
true_intensity = 5e-11

In [3]:
coordinates = vd.grid_coordinates(
    region=[-100, 100, -100, 100],
    spacing=2,
    extra_coords=5,
)

true_moment = hm.magnetic_angles_to_vec(
    inclination=true_inclination,
    declination=true_declination,
    intensity=true_intensity,
)

data = dipole_bz(coordinates, dipole_coordinates, true_moment)

print(dipole_coordinates)
print(true_moment)

linear_model = MagneticMomentBz((499, 499, -14))
linear_model.fit(coordinates, data)
print(linear_model.dipole_moment_)

(500, 500, -15)
(np.float64(2.783351996132097e-11), np.float64(3.3170697408446925e-11), np.float64(-2.4999999999999998e-11))
[ 3.02179525e-11  3.57779743e-11 -2.49737722e-11]


In [4]:
# Initial guess
location = np.array([495, 495, -10])
moment = linear_model.dipole_moment_
print(location)
print(moment)

[495 495 -10]
[ 3.02179525e-11  3.57779743e-11 -2.49737722e-11]


In [5]:
def _jacobian_kernel(x, y, z, xc, yc, zc, mx, my, mz, result):
    factor = choclo.constants.VACUUM_MAGNETIC_PERMEABILITY / (4 * np.pi)

    for i in numba.prange(x.size):
        dx = x[i] - xc
        dy = y[i] - yc
        dz = z[i] - zc

        r2 = dx**2 + dy**2 + dz**2
        r5 = r2 ** (5 / 2)
        r7 = r2 ** (7 / 2)

        # ∂bz / ∂xc
        dBz_dxc = factor * (
            (15 * my * dy * dz * dx) / r7
            + mz * ((15 * dz**2 * dx) / r7 - (3 * dx) / r5)
            + (15 * mx * dz * dx**2) / r7
            - (3 * mx * dz) / r5
        )

        # ∂bz / ∂yc
        dBz_dyc = factor * (
            (15 * mx * dx * dz * dy) / r7
            + mz * ((15 * dz**2 * dy) / r7 - (3 * dy) / r5)
            + (15 * my * dz * dy**2) / r7
            - (3 * my * dz) / r5
        )

        # ∂bz / ∂zc
        dBz_dzc = factor * (
            mz * ((15 * dz**3) / r7 - (9 * dz) / r5)
            + (15 * my * dy * dz**2) / r7
            - (3 * my * dy) / r5
            + (15 * mx * dx * dz**2) / r7
            - (3 * mx * dx) / r5
        )

        result[i, 0] = dBz_dxc
        result[i, 1] = dBz_dyc
        result[i, 2] = dBz_dzc
        
jacobian_kernel_jit = numba.jit(_jacobian_kernel, nopython=True, parallel=True)


In [6]:
class NonlinearMagneticDipoleBz:
    def __init__(self, location, dipole_moment):
        self.location = location
        self.dipole_moment = dipole_moment
        self.location_ = None
        self.dipole_moment_ = None
        self.parameters_ = None
        self.jacobian = None

        self.scale_location = 1e-6
        self.scale_moment = 1e-10

    def fit(self,
        coordinates,
        data,
        max_iter=100,
        tol=1e-10,
        lambda_init=1e-2,
        lambda_scale=10.0,
    ):
        coordinates, data = check_fit_input(coordinates, data)
        misfit = []

        lambda_ = lambda_init
        position = self.location
        for _ in range(max_iter):
            linear_model = MagneticMomentBz(position)
            linear_model.fit(coordinates, data)
            moment = linear_model.dipole_moment_

            predicted_data = dipole_bz(coordinates, position, moment)
            residual = data - predicted_data
            current_misfit = np.linalg.norm(residual) ** 2
            misfit.append(current_misfit)

            A = np.empty((data.size, 3))

            x, y, z = coordinates_micrometer_to_meter(coordinates)
            x = x.ravel()
            y = y.ravel()
            z = z.ravel()

            xc, yc, zc = coordinates_micrometer_to_meter(dipole_coordinates)

            _jacobian_kernel(
                x,
                y,
                z,
                xc,
                yc,
                zc,
                moment[0],
                moment[1],
                moment[2],
                A,
            )

            JTJ = A.T @ A
            JTr = A.T @ (residual)

            accepted = False

            for _ in range(50):
                delta = np.linalg.solve(JTJ + lambda_ * np.eye(3), JTr)
                trial_position = position + delta

                linear_model = MagneticMomentBz(trial_position)
                linear_model.fit(coordinates, data)
                trial_moment = linear_model.dipole_moment_
                trial_pred = dipole_bz(coordinates, trial_position, trial_moment)
                trial_residual = data - trial_pred
                trial_misfit = np.linalg.norm(trial_residual) ** 2

                if trial_misfit < current_misfit:
                    position = trial_position
                    lambda_ /= lambda_scale
                    misfit.append(trial_misfit)
                    accepted = True
                    break
                lambda_ *= lambda_scale

            if not accepted:
                break

            if abs(misfit[-1] - misfit[-2]) / misfit[-2] < tol:
                break

        return position, moment


In [7]:
def Gauss_Newton_Levenberg_Marquardt(
    coordinates,
    data,
    position,
    max_iter=100,
    tol=1e-10,
    lambda_init=1e-2,
    lambda_scale=10.0,
):
    coordinates, data = check_fit_input(coordinates, data)
    misfit = []

    lambda_ = lambda_init

    for _ in range(max_iter):
        linear_model = MagneticMomentBz(position)
        linear_model.fit(coordinates, data)
        moment = linear_model.dipole_moment_

        predicted_data = dipole_bz(coordinates, position, moment)
        residual = data - predicted_data
        current_misfit = np.linalg.norm(residual) ** 2
        misfit.append(current_misfit)

        A = np.empty((data.size, 3))

        x, y, z = coordinates_micrometer_to_meter(coordinates)
        x = x.ravel()
        y = y.ravel()
        z = z.ravel()

        xc, yc, zc = coordinates_micrometer_to_meter(dipole_coordinates)

        _jacobian_kernel(
            x,
            y,
            z,
            xc,
            yc,
            zc,
            moment[0],
            moment[1],
            moment[2],
            A,
        )

        JTJ = A.T @ A
        JTr = A.T @ (residual)

        accepted = False

        for _ in range(50):
            delta = np.linalg.solve(JTJ + lambda_ * np.eye(3), JTr)
            trial_position = position + delta

            linear_model = MagneticMomentBz(trial_position)
            linear_model.fit(coordinates, data)
            trial_moment = linear_model.dipole_moment_
            trial_pred = dipole_bz(coordinates, trial_position, trial_moment)
            trial_residual = data - trial_pred
            trial_misfit = np.linalg.norm(trial_residual) ** 2

            if trial_misfit < current_misfit:
                position = trial_position
                lambda_ /= lambda_scale
                misfit.append(trial_misfit)
                accepted = True
                break
            lambda_ *= lambda_scale

        if not accepted:
            break

        if abs(misfit[-1] - misfit[-2]) / misfit[-2] < tol:
            break

    return position, moment

In [8]:
location_, moment_ = Gauss_Newton_Levenberg_Marquardt(coordinates, data, location)
# nonlinear_model = NonlinearMagneticDipoleBz()

print(dipole_coordinates)
print(true_moment)
print(location_)
print(moment_)

(500, 500, -15)
(np.float64(2.783351996132097e-11), np.float64(3.3170697408446925e-11), np.float64(-2.4999999999999998e-11))
[499.51201786 499.33496616  -8.45968024]
[ 4.29768156e-11  5.10953217e-11 -2.50234477e-11]


In [9]:
nonlinear_model = NonlinearMagneticDipoleBz(location, moment)
nl_location_, nl_moment_ = nonlinear_model.fit(coordinates, data)

In [10]:
print(dipole_coordinates)
print(true_moment)
print(nl_location_)
print(nl_moment_)

(500, 500, -15)
(np.float64(2.783351996132097e-11), np.float64(3.3170697408446925e-11), np.float64(-2.4999999999999998e-11))
[499.51201786 499.33496616  -8.45968024]
[ 4.29768156e-11  5.10953217e-11 -2.50234477e-11]
